In [19]:
import pandas as pd
import glob
import os
import tqdm
import gc
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from tqdm import tqdm

In [20]:
f='datasets/athletes_sochi.txt'
dfs = pd.read_csv(f).dropna()

In [21]:
dfs = pd.read_csv(f).dropna()

In [22]:
dfs["BMI"] = dfs["weight"] / (dfs["height"]**2)


In [23]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

vars = ["sport","country"]
# Convert categorical variables to numeric using LabelEncoder
le = LabelEncoder()
for var in vars:
    dfs[var] = le.fit_transform(dfs[var])

# One-hot encode categorical variables
ohe = OneHotEncoder()
ohe.fit(dfs[vars])
encoded_data = ohe.transform(dfs[vars]).toarray()

In [24]:
count_male = 0
count_female = 0
for i in range(len(dfs)-1):
   try:
    if dfs['gender'][i] =='Male' and dfs['total_medals'][i] >0:
        count_male += 1
    elif dfs['gender'][i] =='Female' and dfs['total_medals'][i] >0:
        count_female += 1
        
   except:
    pass
print(f"count male:{count_male}\ncount female:{count_female}")
        

count male:85
count female:70


In [25]:
alldata = dfs[ ['weight', 'height', 'BMI','total_medals',"sport","country", 'gender'] ]


In [26]:
alldata = alldata.sample(frac=1)

In [27]:
allinput = alldata[ ['weight', 'height','total_medals',"sport","country", 'BMI'] ]
alloutput = alldata["gender"].apply(lambda x: int(x == "Male"))

In [28]:
allinput

,weight,height,total_medals,sport,country,BMI
48,63.0,1.71,0,5,63,21.545091
440,66.0,1.75,0,8,79,21.551020
1454,92.0,1.85,0,6,55,26.880935
71,75.0,1.84,0,3,12,22.152647
621,85.0,1.88,0,7,26,24.049344
...,...,...,...,...,...,...
2463,59.0,1.70,0,3,79,20.415225
2200,65.0,1.73,0,12,54,21.718066
1791,60.0,1.73,0,8,24,20.047446
1207,66.0,1.75,0,9,15,21.551020


In [29]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(allinput, alloutput, test_size=0.2,random_state=101)

In [30]:
model = linear_model.LogisticRegression(tol=1e-6,max_iter=1000)

In [31]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000, tol=1e-06)

In [32]:
Y_test_predicted = model.predict(X_test)

In [33]:
[tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
confusion_matrix(Y_test, Y_test_predicted)

array([[169,  28],
       [ 29, 270]], dtype=int64)

In [34]:
precision = tp / (tp + fp)
recall = tp / tp + fn

print("Precision: ", tp / (tp + fp))
print("Recall: ", tp / (tp + fn))
print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
print("F1: ",  (precision * recall) / ((precision) + recall))

Precision:  0.9060402684563759
Recall:  0.903010033444816
Accuracy:  0.8850806451612904
F1:  0.8794788273615635


In [35]:
from sklearn.metrics import classification_report


In [36]:
report = classification_report(Y_test, model.predict(X_test), target_names=['Male', 'Female'])
print(report)

              precision    recall  f1-score   support

        Male       0.85      0.86      0.86       197
      Female       0.91      0.90      0.90       299

    accuracy                           0.89       496
   macro avg       0.88      0.88      0.88       496
weighted avg       0.89      0.89      0.89       496

